# 01 - Fine tune the Microsoft Phi3 model to create MaximusLLM

This notebook aims to fine tune the Phi3 Mini Instruct 128k model from Microsoft to create a custom model named MaximusLLM.
The fine-tuning process will be conducted using the Hugging Face Transformers library and the PEFT library for efficient training.

## a) Import the required libraries

The following libraries are requiredered to fine-tune the model:

- ***sys***: Provides access to some variables used or maintained by the Python interpreter and to functions that interact strongly with the interpreter.
- ***logging***: Allows you to emit messages to a log file or to the system console.
- ***datasets***: Provides a simple and efficient way to load, preprocess, and share datasets.
- ***huggingface_hub***: Offers a command-line interface to the Hugging Face Hub, allowing you to upload and download models, datasets, and other artifacts.
- ***peft***: Enables parameter-efficient fine-tuning of large language models, reducing the computational resources required for training.
- ***torch***: A popular machine learning library that provides a wide range of functionalities for building and training neural networks.
- ***transformers***: A library for state-of-the-art natural language processing (NLP) models, providing easy-to-use interfaces for tasks such as text classification, translation, and question answering.
- ***trl***: A library for training reinforcement learning agents in NLP tasks, such as dialogue generation and sentiment analysis.


In [38]:
import os
import sys
import logging
import dotenv
import datasets
from datasets import load_dataset
from datasets import DatasetDict
from huggingface_hub import notebook_login
from peft import LoraConfig
import torch
import wandb
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft.utils.save_and_load import get_peft_model_state_dict
from peft import PeftModel

Now import the environment variables containing the various tokens.

In [2]:
dotenv.load_dotenv()

True

And set the CUDA_HOME varaible for the NVidia GPU libraries

In [3]:
os.environ["CUDA_HOME"]="/usr"

## b) Define the fine tuning parameters and configurations

This is a configuration dictionary for fine-tuning the Microsoft Phi3 model. It includes settings for training, evaluation, learning rate, logging, and saving checkpoints. The training uses bf16 precision, a cosine learning rate scheduler, and gradient checkpointing for memory efficiency.

In [4]:
training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    }

This is a configuration dictionary for fine-tuning a Microsoft Phi3 model using Parameter-Efficient Fine-Tuning (PEFT) with Low-Rank Adaptation (LoRA). It specifies the rank (r) of the update matrices, the scaling factor (lora_alpha), dropout rate, bias handling, task type, target modules for adaptation, and modules to save during the fine-tuning process.

In [5]:
peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}

The code sets up training configurations for a model using Hugging Face's Transformers library, including parameters for training and Low-Rank Adaptation (LoRA) fine-tuning.

In [6]:
train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

## c) Setting up the logging for the fine tuining process

This logger will be used to log messages at various levels such as debug, info, warning, error, and critical. 

In [7]:
logger = logging.getLogger(__name__)

In [8]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

The code snippet sets the logging level for various libraries (datasets, transformers) and the logger to the same level specified in the training configuration. This ensures consistent and appropriate verbosity across the different components of the software, making it easier to debug and understand the execution flow.

In [9]:
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

The code snippet logs information about the training process, including the process rank, device, and whether distributed training or 16-bit training is being used. It also logs the training/evaluation parameters and PEFT parameters for reference.

In [10]:
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

2024-07-08 20:46:55 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1 distributed training: True, 16-bits training: False
2024-07-08 20:46:55 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,

## d) Load the Phi3 model and tokenizer

The Microsoft Phi-3 model is a large language model developed by Microsoft. It's a part of the Phi series, which includes models like Phi-1 and Phi-2. The "Phi-3-mini-128k-instruct" is a variant of the Phi-3 model that has been fine-tuned on a smaller dataset of 128,000 tokens. This makes it more computationally efficient and faster to train, while still maintaining a good level of performance. The "-instruct" in the name suggests that this model has been trained to follow instructions, making it suitable for tasks like text generation and question answering. However, without more specific information, I can't provide a detailed description of its capabilities or limitations.

In [11]:
hd_model_name = "microsoft/Phi-3-mini-128k-instruct"

The code sets up parameters for loading a Microsoft Phi3 model, including using flash attention for faster computation, using bfloat16 data type for memory efficiency, and not using cache or device mapping.

In [12]:
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    #attn_implementation="flash_attention_2",  # loading the model with flash-attention support
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map=None
)

The code snippet loads a pre-trained causal language model and its corresponding tokenizer from a specified checkpoint path. This is a common step in fine-tuning a model for a specific task.

In [13]:
model = AutoModelForCausalLM.from_pretrained(hd_model_name, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(hd_model_name)

[INFO|configuration_utils.py:733] 2024-07-08 20:46:55,947 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:733] 2024-07-08 20:46:56,073 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 20:46:56,075 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop"

2024-07-08 20:46:56 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - `flash-attention` package not found, consider installing for better performance: /home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda9SetDeviceEi.
2024-07-08 20:46:56 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


[INFO|modeling_utils.py:3556] 2024-07-08 20:46:56,382 >> loading weights file model.safetensors from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/model.safetensors.index.json
[INFO|modeling_utils.py:1531] 2024-07-08 20:46:56,384 >> Instantiating Phi3ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1000] 2024-07-08 20:46:56,384 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 32000,
  "use_cache": false
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4364] 2024-07-08 20:46:57,559 >> All model checkpoint weights were used when initializing Phi3ForCausalLM.

[INFO|modeling_utils.py:4372] 2024-07-08 20:46:57,560 >> All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-128k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:955] 2024-07-08 20:46:57,674 >> loading configuration file generation_config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/generation_config.json
[INFO|configuration_utils.py:1000] 2024-07-08 20:46:57,674 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

[INFO|tokenization_utils_base.py:2161] 2024-07-08 20:46

This code sets the maximum token length to 2048, uses the unknown token for padding to prevent endless generation, adjusts the padding token ID, and sets the padding side to the right for the tokenizer in a machine learning model.

In [14]:
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

## e) Prepare the training and validation datasets

This function takes a conversation-style example and a tokenizer as input, formats the messages in the example using the chat template provided by the tokenizer, and adds the formatted string to the example dictionary under the key "text".


In [15]:
def apply_chat_template(example,tokenizer):
    """
    This function takes a conversation-style example and a tokenizer as input,
    formats the messages in the example using the chat template provided by the tokenizer,
    and adds the formatted string to the example dictionary under the key "text".

    Parameters:
    example (dict): A dictionary containing a key "messages" with a list of messages.
    tokenizer (object): An object with a method apply_chat_template that formats a list of messages into a single string.

    Returns:
    dict: The modified example dictionary with a new key "text" containing the formatted string.
    """
    messages = example["messages"]
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False)
    return example

The code loads a dataset and prints the number of samples in the 'train', 'eval', and 'test' subsets. It shows the dataset's size distribution.

In [16]:
raw_dataset = load_dataset("awels/maximo_admin_dataset")
len(raw_dataset['train'])

Overwrite dataset info from restored data version if exists.


2024-07-08 20:47:00 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.


Loading Dataset info from /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65


2024-07-08 20:47:00 - INFO - datasets.info - Loading Dataset info from /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65


Found cached dataset maximo_admin_dataset (/home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65)


2024-07-08 20:47:00 - INFO - datasets.builder - Found cached dataset maximo_admin_dataset (/home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65)


Loading Dataset info from /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65


2024-07-08 20:47:00 - INFO - datasets.info - Loading Dataset info from /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65


12687

Now prepare the training and validation datasets.

In [17]:
# Diviser le dataset d'entraînement en deux splits : train et test
train_test_split = raw_dataset["train"].train_test_split(test_size=0.2)  # 20% pour le test, 80% pour l'entraînement

# Créer un DatasetDict avec les nouveaux splits
new_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

train_dataset = new_dataset["train"]
test_dataset = new_dataset["test"]

print ("Train set : " + str(len(train_dataset)))
print ("Test set : " + str(len(test_dataset)))

Caching indices mapping at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-303a2a11d701da26.arrow


2024-07-08 20:47:00 - INFO - datasets.arrow_dataset - Caching indices mapping at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-303a2a11d701da26.arrow


Caching indices mapping at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-c9ef3ef2eecaae0d.arrow


2024-07-08 20:47:00 - INFO - datasets.arrow_dataset - Caching indices mapping at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-c9ef3ef2eecaae0d.arrow
Train set : 10149
Test set : 2538


In [18]:
train_dataset

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 10149
})

The code applies a chat template to the training dataset using a tokenizer, with parallel processing for efficiency, and removes unnecessary columns. Do the same for test datasets as well.

In [19]:
column_names = list(train_dataset.features)

processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to train",
)

processed_test_dataset = test_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to test",
)

Process #0 will write at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-01908981a16589d6_00000_of_00010.arrow


2024-07-08 20:47:01 - INFO - datasets.arrow_dataset - Process #0 will write at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-01908981a16589d6_00000_of_00010.arrow


Process #1 will write at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-01908981a16589d6_00001_of_00010.arrow


2024-07-08 20:47:01 - INFO - datasets.arrow_dataset - Process #1 will write at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-01908981a16589d6_00001_of_00010.arrow


Process #2 will write at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-01908981a16589d6_00002_of_00010.arrow


...

2024-07-08 20:47:03 - INFO - datasets.arrow_dataset - Caching processed dataset at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-e72d48380e23bb03_00008_of_00010.arrow


Caching processed dataset at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-e72d48380e23bb03_00009_of_00010.arrow


2024-07-08 20:47:03 - INFO - datasets.arrow_dataset - Caching processed dataset at /home/franck/.cache/huggingface/datasets/awels___maximo_admin_dataset/default/0.0.0/91b3b084749da7d748f7750b5d350c582dd7ac65/cache-e72d48380e23bb03_00009_of_00010.arrow


Concatenating 10 shards


2024-07-08 20:47:04 - INFO - datasets.arrow_dataset - Concatenating 10 shards


## f) Execute the training of the MaximusLLM model

The code is using the SFTTrainer (Supervised Fine-Tuning Trainer) to fine-tune the Microsoft Phi3 model. It sets up the model, training configuration, and dataset for training and evaluation. The maximum sequence length is set to 64, and the trainer is configured to pack sequences efficiently.

In [20]:
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_test_dataset,
    max_seq_length=64,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True
)

/home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
[INFO|training_args.py:2048] 2024-07-08 20:47:04,171 >> PyTorch: setting up devices
/home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the

2024-07-08 20:47:04 - INFO - datasets.builder - Using custom data configuration default-2a050f9e3c68d40b


Loading Dataset Infos from /home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/datasets/packaged_modules/generator


2024-07-08 20:47:04 - INFO - datasets.info - Loading Dataset Infos from /home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/datasets/packaged_modules/generator


Generating dataset generator (/home/franck/.cache/huggingface/datasets/generator/default-2a050f9e3c68d40b/0.0.0)


...

Unable to verify splits sizes.


2024-07-08 20:47:06 - INFO - datasets.utils.info_utils - Unable to verify splits sizes.


Dataset generator downloaded and prepared to /home/franck/.cache/huggingface/datasets/generator/default-b9b796c42d54b53c/0.0.0. Subsequent calls will reuse this data.


2024-07-08 20:47:06 - INFO - datasets.builder - Dataset generator downloaded and prepared to /home/franck/.cache/huggingface/datasets/generator/default-b9b796c42d54b53c/0.0.0. Subsequent calls will reuse this data.


[INFO|trainer.py:642] 2024-07-08 20:47:07,436 >> Using auto half precision backend


This code modifies the state_dict method of a PyTorch model to use the get_peft_model_state_dict function from the PEFT library. This is done to fine-tune the Microsoft Phi3 model. If the Torch version is 2 or higher and the platform is not Windows, the model is then compiled for performance optimization.

In [21]:
original_state_dict = model.state_dict

def custom_state_dict(self, *args, **kwargs):
    return get_peft_model_state_dict(self, original_state_dict(*args, **kwargs))

model.state_dict = custom_state_dict.__get__(model, type(model))

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

Now let's train the model. PLease note that login to WanDB will be required

In [22]:
wandb.login(key=os.environ['WANDB_API_KEY'])

2024-07-08 20:47:08 - ERROR - wandb.jupyter - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: fbeawels (fbeawels-awels-engineering). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/franck/.netrc


True

In [23]:
train_result = trainer.train()

[INFO|trainer.py:2128] 2024-07-08 20:47:09,463 >> ***** Running training *****
[INFO|trainer.py:2129] 2024-07-08 20:47:09,463 >>   Num examples = 12,499
[INFO|trainer.py:2130] 2024-07-08 20:47:09,463 >>   Num Epochs = 1
[INFO|trainer.py:2131] 2024-07-08 20:47:09,464 >>   Instantaneous batch size per device = 4
[INFO|trainer.py:2134] 2024-07-08 20:47:09,464 >>   Total train batch size (w. parallel, distributed & accumulation) = 4
[INFO|trainer.py:2135] 2024-07-08 20:47:09,464 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2136] 2024-07-08 20:47:09,464 >>   Total optimization steps = 3,125
[INFO|trainer.py:2137] 2024-07-08 20:47:09,466 >>   Number of trainable parameters = 25,165,824
[INFO|integration_utils.py:750] 2024-07-08 20:47:09,471 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a dif

  0%|          | 0/3125 [00:00<?, ?it/s]

2024-07-08 20:47:13 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - You are not running the flash-attention implementation, expect numerical differences.
{'loss': 6.7887, 'grad_norm': 10.1875, 'learning_rate': 1.6e-07, 'epoch': 0.01}
{'loss': 6.5376, 'grad_norm': 6.0, 'learning_rate': 3.2e-07, 'epoch': 0.01}
{'loss': 6.2477, 'grad_norm': 12.875, 'learning_rate': 4.800000000000001e-07, 'epoch': 0.02}
{'loss': 6.5146, 'grad_norm': 8.0625, 'learning_rate': 6.4e-07, 'epoch': 0.03}


[INFO|trainer.py:3478] 2024-07-08 20:47:31,827 >> Saving model checkpoint to ./checkpoint_dir/checkpoint-100


{'loss': 6.179, 'grad_norm': 4.8125, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.03}


...

{'loss': 2.3465, 'grad_norm': 1.921875, 'learning_rate': 1.2335990856710001e-09, 'epoch': 0.99}


[INFO|configuration_utils.py:733] 2024-07-08 20:56:44,131 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 20:56:44,132 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_va

{'loss': 2.2627, 'grad_norm': 2.109375, 'learning_rate': 4.934785965721167e-11, 'epoch': 1.0}


[INFO|trainer.py:3478] 2024-07-08 20:56:48,946 >> Saving model checkpoint to ./checkpoint_dir/checkpoint-3125
[INFO|configuration_utils.py:733] 2024-07-08 20:56:49,187 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 20:56:49,188 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embe

{'train_runtime': 580.1929, 'train_samples_per_second': 21.543, 'train_steps_per_second': 5.386, 'train_loss': 3.0464051943969728, 'epoch': 1.0}


This snippet retrieves training metrics, logs them for monitoring, and saves them for future reference. It's a common practice in machine learning to log and save metrics to understand and compare model performance.

In [24]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** train metrics *****
  epoch                    =        1.0
  total_flos               = 16752384GF
  train_loss               =     3.0464
  train_runtime            = 0:09:40.19
  train_samples_per_second =     21.543
  train_steps_per_second   =      5.386


In [25]:
# Save the state of the trainer
trainer.save_state() 

## g) Evaluate the new model

The code sets the padding side of the tokenizer to 'left', evaluates the model using a trainer, calculates the number of evaluation samples, logs and saves the evaluation metrics. This is a common step in fine-tuning a model like Microsoft Phi3.

In [27]:
tokenizer.padding_side = 'left'
metrics = trainer.evaluate()
metrics["eval_samples"] = len(processed_test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

[INFO|trainer.py:3788] 2024-07-08 21:16:31,274 >> 
***** Running Evaluation *****
[INFO|trainer.py:3790] 2024-07-08 21:16:31,274 >>   Num examples = 3042
[INFO|trainer.py:3793] 2024-07-08 21:16:31,274 >>   Batch size = 4


  0%|          | 0/761 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        1.0
  eval_loss               =     2.4107
  eval_runtime            = 0:00:31.07
  eval_samples            =       2538
  eval_samples_per_second =     97.883
  eval_steps_per_second   =     24.487


And now save it locally before proceeding to preparation before upload to HF hub

In [28]:
trainer.save_model(train_conf.output_dir)

[INFO|trainer.py:3478] 2024-07-08 21:18:15,955 >> Saving model checkpoint to ./checkpoint_dir
[INFO|configuration_utils.py:733] 2024-07-08 21:18:16,247 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 21:18:16,248 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,

## h) Transform the model to GGUF 8-bit quantized format and load it to Hugging Face Hub    

In [29]:
!cd ../modules/ && git clone https://github.com/ggerganov/llama.cpp.git

Clonage dans 'llama.cpp'...
remote: Enumerating objects: 29214, done.
remote: Counting objects: 100% (8439/8439), done.
remote: Compressing objects: 100% (577/577), done.
remote: Total 29214 (delta 8177), reused 7869 (delta 7862), pack-reused 20775
Réception d'objets: 100% (29214/29214), 50.89 Mio | 14.83 Mio/s, fait.
Résolution des deltas: 100% (21008/21008), fait.


In [30]:
!pip install -r ../modules/llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
  Using cached sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 8.3 MB/s eta 0:00:0000:0100:01
Using cached sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mistral-common 1.2.1 requires sentencepiece==0.1.99, but you have sent

The code fine-tunes the Microsoft Phi3 model using a custom adapter model. It loads the base model and the adapter, then merges them into a new model, which is then unloaded from memory to save resources.

In [50]:
base_model_name = "microsoft/Phi-3-mini-128k-instruct"
adapter_model_name = "./checkpoint_dir"

model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(base_model_name, **model_kwargs)

model = PeftModel.from_pretrained(model, adapter_model_name)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

[INFO|configuration_utils.py:733] 2024-07-08 21:46:12,253 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:733] 2024-07-08 21:46:12,459 >> loading configuration file config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 21:46:12,460 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop"

2024-07-08 21:46:12 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - `flash-attention` package not found, consider installing for better performance: /home/franck/Applications/miniconda3/envs/MaximusLLM/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda9SetDeviceEi.
2024-07-08 21:46:12 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


[INFO|modeling_utils.py:3556] 2024-07-08 21:46:12,584 >> loading weights file model.safetensors from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/model.safetensors.index.json
[INFO|modeling_utils.py:1531] 2024-07-08 21:46:12,586 >> Instantiating Phi3ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1000] 2024-07-08 21:46:12,591 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 32000,
  "use_cache": false
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4364] 2024-07-08 21:46:13,912 >> All model checkpoint weights were used when initializing Phi3ForCausalLM.

[INFO|modeling_utils.py:4372] 2024-07-08 21:46:13,913 >> All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-128k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:955] 2024-07-08 21:46:14,022 >> loading configuration file generation_config.json from cache at /home/franck/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/generation_config.json
[INFO|configuration_utils.py:1000] 2024-07-08 21:46:14,023 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

[INFO|tokenization_utils_base.py:2161] 2024-07-08 21:46

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3FlashAttention2(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              

In [47]:
new_model = model.merge_and_unload()


In [51]:
new_model.save_pretrained("./phi3-128k-3b-v0.1")
tokenizer.save_pretrained("./phi3-128k-3b-v0.1")

[INFO|configuration_utils.py:472] 2024-07-08 21:46:37,438 >> Configuration saved in ./phi3-128k-3b-v0.1/config.json
[INFO|configuration_utils.py:769] 2024-07-08 21:46:37,439 >> Configuration saved in ./phi3-128k-3b-v0.1/generation_config.json
[INFO|modeling_utils.py:2698] 2024-07-08 21:47:07,422 >> The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./phi3-128k-3b-v0.1/model.safetensors.index.json.
[INFO|tokenization_utils_base.py:2574] 2024-07-08 21:47:07,424 >> tokenizer config file saved in ./phi3-128k-3b-v0.1/tokenizer_config.json
[INFO|tokenization_utils_base.py:2583] 2024-07-08 21:47:07,424 >> Special tokens file saved in ./phi3-128k-3b-v0.1/special_tokens_map.json


('./phi3-128k-3b-v0.1/tokenizer_config.json',
 './phi3-128k-3b-v0.1/special_tokens_map.json',
 './phi3-128k-3b-v0.1/tokenizer.model',
 './phi3-128k-3b-v0.1/added_tokens.json',
 './phi3-128k-3b-v0.1/tokenizer.json')

In [52]:
!python ../modules/llama.cpp/convert_hf_to_gguf.py phi3-128k-3b-v0.1 \
  --outfile ../models/maximusLLM-phi3-128k-3b-v0.1.gguf \
  --outtype f16

INFO:hf-to-gguf:Loading model: phi3-128k-3b-v0.1
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:Set model tokenizer
INFO:gguf.vocab:Setting special token type bos to 1
INFO:gguf.vocab:Setting special token type eos to 32000
INFO:gguf.vocab:Setting special token type unk to 0
INFO:gguf.vocab:Setting special token type pad to 32000
INFO:gguf.vocab:Setting add_bos_token to False
INFO:gguf.vocab:Setting add_eos_token to False
INFO:gguf.vocab:Setting chat_template to {% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf

Now let's finish by uploading the model to Hugging Face Hub

In [54]:
from huggingface_hub import HfApi
api = HfApi(token=os.environ['HF_TOKEN'])

In [56]:
model_id = "awels/maximusLLM-3b-128k-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")

RepoUrl('https://huggingface.co/awels/maximusLLM-3b-128k-gguf', endpoint='https://huggingface.co', repo_type='model', repo_id='awels/maximusLLM-3b-128k-gguf')

In [61]:
api.upload_file(
    path_or_fileobj="../models/maximusLLM-phi3-128k-3b-v0.1.gguf",
    path_in_repo="maximusLLM-phi3-128k-3b-v0.1.gguf",
    repo_id=model_id,
)

maximusLLM-phi3-128k-3b-v0.1.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/awels/maximusLLM-3b-128k-gguf/commit/f3ec0e77b755d375849a3465632bcffb13d23f2c', commit_message='Upload maximusLLM-phi3-128k-3b-v0.1.gguf with huggingface_hub', commit_description='', oid='f3ec0e77b755d375849a3465632bcffb13d23f2c', pr_url=None, pr_revision=None, pr_num=None)